# Pattern Recognition - Flow S
## Pre-Lab 3: Genre and Emotion Recognition from Music
### Dimitris Dimos - 031 17 165
### Konstantinos Kopsinis - 031 17 062

In [2]:
## packages
import numpy as np
import pandas as pd
import os
import gc
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from librosa.display import specshow
from IPython.display import clear_output
from torch.nn.functional import relu

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

from sklearn.metrics import classification_report

## Step 0: Familiarization with Kaggle kernel

In [122]:
## print data after having loaded it
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
clear_output()

In [123]:
# explore subfiles
os.listdir("/kaggle/input/patreco3-multitask-affective-music/data/")

In [3]:
# data directories
fma_genre_spectrograms = "/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/"
fma_genre_spectrograms_beat = "/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/"
multitask_dataset = "/kaggle/input/patreco3-multitask-affective-music/data/multitask_dataset/"
multitask_dataset_beat = "/kaggle/input/patreco3-multitask-affective-music/data/multitask_dataset_beat/"

# print contents
os.listdir(fma_genre_spectrograms)

## Step 1: Familiarization with Mel Spectrograms

In [125]:
## (a) randomly choose two FMA samples
with open(fma_genre_spectrograms + 'train_labels.txt') as f:
    lines = f.readlines()
    n_lines = len(lines) - 1 # the first line contains headers
    
    # randomly choose two lines
    line0 = 1336 # np.random.randint(1, n_lines)
    line1 = 2346 # np.random.randint(1, n_lines)
    
    file0, label0 = lines[line0].split()
    file1, label1 = lines[line1].split()
    
    print(file0 +'\t\t'+ label0)
    print(file1 +'\t\t'+ label1)

In [126]:
## (b) read files - print spectrograms
spec0 = np.load(fma_genre_spectrograms + 'train/' + file0.strip(".gz"))
spec1 = np.load(fma_genre_spectrograms + 'train/' + file1.strip(".gz"))

print(f'(mel + chroma frequencies, time steps) = {spec0.shape}')  # (mel + chroma freqs x time steps)
print(f'(mel + chroma frequencies, time steps) = {spec1.shape}')  # (mel + chroma freqs x time steps)

# decompose into spectrogram and chromagram
mel0, chroma0 = spec0[:128], spec0[128:]
mel1, chroma1 = spec1[:128], spec1[128:]

In [127]:
## (c) plot spectrograms
plt.rcParams['figure.figsize'] = [20, 6]

# spectrograms
plt.subplot(1, 2, 1)
plt.title(label0)
specshow(mel0)
plt.subplot(1, 2, 2)
plt.title(label1)
specshow(mel1)
plt.show()

## Step 2: Beat-synced Spectrograms

In [128]:
## (a) 

print(f"1st spectrogram shape: {mel0.shape}")
print(f"2nd spectrogram shape: {mel1.shape}")
print(f'time steps: {mel0.shape[1]}')

In [129]:
## (b) we now repeat the same for the beat-synced spectrograms

# read files - print spectrograms
spec0_beat = np.load(fma_genre_spectrograms_beat + 'train/' + file0.strip(".gz"))
spec1_beat = np.load(fma_genre_spectrograms_beat + 'train/' + file1.strip(".gz"))

# decompose into spectrogram and chromagram
mel0_beat, chroma0_beat = spec0_beat[:128], spec0_beat[128:]
mel1_beat, chroma1_beat = spec1_beat[:128], spec1_beat[128:]

print(f'1st spectrogram shape (beat-sync): {mel0_beat.shape}')  # (mel freqs x time steps)
print(f'2nd spectrogram shape (beat-sync): {mel1_beat.shape}')  # (mel freqs x time steps)

## plot spectrograms
plt.rcParams['figure.figsize'] = [20, 6]

plt.subplot(1, 2, 1)
plt.title(label0)
specshow(mel0_beat)
plt.subplot(1, 2, 2)
plt.title(label1)
specshow(mel1_beat)
plt.show()

## Step 3: Familiarization with Chromagrams

In [130]:
## plots
plt.rcParams['figure.figsize'] = [20, 6]

plt.subplot(1, 2, 1)
plt.title(label0)
specshow(chroma0)
plt.subplot(1, 2, 2)
plt.title(label1)
specshow(chroma1)
plt.show()

In [131]:
# shapes
print(f"1st chromagram shape: {chroma0.shape}")
print(f"2nd chromagram shape: {chroma1.shape}")
print(f'time steps: {chroma0.shape[1]}')

In [132]:
# beat-synced chromagrams

print(f'1st chromagram shape (beat-sync): {chroma0_beat.shape}')  # (chroma freqs x time steps)
print(f'2nd chromagram shape (beat-sync): {chroma1_beat.shape}')  # (chroma freqs x time steps)

## plot spectrograms
plt.rcParams['figure.figsize'] = [20, 6]

plt.subplot(1, 2, 1)
plt.title(label0)
specshow(chroma0_beat)
plt.subplot(1, 2, 2)
plt.title(label1)
specshow(chroma1_beat)
plt.show()

## Step 4: Data Loading and Analysis

In [4]:
# we now import the auxiliary code from github
!cp -r /kaggle/input/lab3-aux/* ./
import dataset
import dataset2
import lab2_lstm

Since our RAM is only 13GB, we have to make some manual arrangements so as to avoid allocating unnecessary space. For this cause, we define our data_generator having two modes:

- mode (a) [forShow=False]: dataloading and returning the necessary dataloaders (allocates RAM)
- mode (b) [forShow=True]: loading only train and val data and showing relevant information (does not allocate RAM)

In [24]:
## data generator
def data_generator(directory, class_mapping, read_spec_fn, batch_size=32, forShow=False):
    train_val_loader = dataset.SpectrogramDataset(directory,
                                                  train=True,
                                                  class_mapping=class_mapping,
                                                  max_length=-1,
                                                  read_spec_fn=read_spec_fn
                                                 )
    # train and validation sets
    train_loader, val_loader = dataset.torch_train_val_split(train_val_loader, batch_size ,batch_size, val_size=.33)
    
    # if we only need to see train data info, return here
    # so as to not allocate any RAM
    if forShow:
        # print info
        datum = next(iter(train_loader))
        print('Data shape')
        print(datum[0].shape)  # shape of data
        print('Labels')
        print(datum[1])  # labels in batch
        print('Lengths')
        print(datum[2])  # length of each element in batch
        return
    
    # test set
    ttest_loader_ = dataset.SpectrogramDataset(directory,
                                               train=False,
                                               class_mapping=class_mapping,
                                               max_length=-1,
                                               read_spec_fn=read_spec_fn
                                              )
    ttest_loader = DataLoader(ttest_loader_, batch_size=batch_size)
    
    return train_loader, val_loader, ttest_loader

At first we will use the function in mode (b), to showcase our data info. Afterwards, when we will train our LSTMs we will use the data_generator in mode (a). This way, we will have the convenience to delete unnecessary data whenever we want to (after each training).

### Load non-beat-synced mel Spectrograms

In [135]:
data_generator(directory=fma_genre_spectrograms, class_mapping=dataset.CLASS_MAPPING, read_spec_fn=dataset.read_mel_spectrogram, forShow=True)

### Load beat-synced mel Spectrograms

In [136]:
data_generator(directory=fma_genre_spectrograms_beat, class_mapping=dataset.CLASS_MAPPING, read_spec_fn=dataset.read_mel_spectrogram, forShow=True)

### Load non-beat-synced Chromagrams

In [137]:
data_generator(directory=fma_genre_spectrograms, class_mapping=dataset.CLASS_MAPPING, read_spec_fn=dataset.read_chromagram, forShow=True)

### Load beat-synced Chromagrams

In [138]:
data_generator(directory=fma_genre_spectrograms_beat, class_mapping=dataset.CLASS_MAPPING, read_spec_fn=dataset.read_chromagram, forShow=True)

### Load 

Spectrogram-Chromagram (non-beat-synced)

In [139]:
data_generator(directory=fma_genre_spectrograms, class_mapping=dataset.CLASS_MAPPING, read_spec_fn=dataset.read_fused_spectrogram, forShow=True)

In [140]:
# let's free some unnecessary RAM
gc.collect()

### Plot Before-After Histograms

In [141]:
# create the same dataset without merging classes
mel_beat_unmerged = dataset.SpectrogramDataset(fma_genre_spectrograms_beat,
                                               train=True,
                                               max_length=-1,
                                               read_spec_fn=dataset.read_mel_spectrogram
                                              )

mel_beat_merged = dataset.SpectrogramDataset(fma_genre_spectrograms_beat,
                                             train=True,
                                             class_mapping=dataset.CLASS_MAPPING,
                                             max_length=-1,
                                             read_spec_fn=dataset.read_mel_spectrogram
                                            )

# collect the labels sequence before
labels_unmerged = []
for mel_spectrum in mel_beat_unmerged:
    labels_unmerged.append(mel_spectrum[1])
    
# collect the labels sequence after
labels_merged = []
for mel_spectrum in mel_beat_merged:
    labels_merged.append(mel_spectrum[1])

In [142]:
# free some space
del mel_beat_unmerged
del mel_beat_merged
gc.collect()

## Step 5: Music Genre Recognition using LSTMs

In [25]:
# hyperparameters
EPOCHS = 80
RNN_SIZE = 32
n_mel = 128
n_chroma = 12
n_classes = 10
num_layers = 1
lr = 1e-3
criterion = nn.CrossEntropyLoss()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
# function to train a model for one epoch
# returns the train loss for this epoch
def one_epoch_train(model, dataloader, criterion, optimizer):
    model.train() # switch to train mode: enable dropout
    overall_loss = 0.0
    DEVICE = next(model.parameters()).device # set DEVICE to the model's predefined device
    
    for idx, batch in enumerate(dataloader, 1):
        
        inputs  = batch[0].to(DEVICE)
        labels  = batch[1].to(DEVICE)
        lengths = batch[2]
        
        optimizer.zero_grad()   # zero gradients out
        y_preds = model(inputs, lengths)   # forward pass
        loss = criterion(y_preds, labels)  # compute loss function
        loss.backward()   # compute gradients
        optimizer.step()  # update parameters

        overall_loss += loss.data.item()

    return overall_loss/idx   # train loss

In [109]:
# function to evaluate model
# returns the loss and the predictions
def evaluate_model(model, dataloader, criterion, regression=False):
    model.eval() # switch to evaluation mode: disable dropout
    overall_loss = 0.0
    DEVICE = next(model.parameters()).device # set DEVICE to the model's predefined device
    
    y_pred = [] # predicted labels
    y_gold = [] # gold labels
    
    with torch.no_grad(): # do not compute gradients
        for idx, batch in enumerate(dataloader, 1):
            
            inputs  = batch[0].to(DEVICE)
            labels  = batch[1].to(DEVICE)
            lengths = batch[2]
                
            y_preds = model(inputs, lengths)  # forward pass
            loss = criterion(y_preds, labels)
            if not regression:
                prediction = torch.argmax(y_preds, dim=1) # predict
            else:
                prediction = y_preds
            
            overall_loss += loss.data.item()
            
            y_pred.append(prediction.cpu().numpy())
            y_gold.append(labels.cpu().numpy())

    return overall_loss/idx, (y_gold, y_pred)

In [28]:
# function to intentionally overfit a model
# and check for bugs and trainability
def overfit_train(model, data_info, criterion, optimizer, epochs):
    model.train()
    DEVICE = next(model.parameters()).device
    
    directory = data_info[0]
    class_mapping = data_info[1]
    read_spec_fn = data_info[2]
    
    train_loader, val_loader, test_loader = data_generator(directory, class_mapping, read_spec_fn)
    train_batch = next(iter(train_loader)) # keep only one batch of 32 samples
    
    for epoch in range(epochs):

        # Get the inputs (batch)
        inputs = train_batch[0].to(DEVICE)
        labels = train_batch[1].to(DEVICE)
        lengths = train_batch[2]
        
        optimizer.zero_grad() # zero gradients out - pytorch accumulates them otherwise
        y_preds = model(inputs, lengths) # forward pass
        loss = criterion(y_preds, labels) # calculate loss
        loss.backward() # calculate gradients
        optimizer.step() # update parameters

        if epoch%500 == 0:
            print(f'Epoch {epoch}: \t Training Loss = {loss.data.item()}')
            
    del train_loader
    del val_loader
    del test_loader
    gc.collect()

In [169]:
# function to train a model for multiple epochs
def train_model(model, data_info, criterion, optimizer, epochs, early_stopping=False, overfit_batch=False, regression=False):
    
    if overfit_batch:
        overfit_train(model, data_info, criterion, optimizer, epochs)
        return
    
    train_losses = []
    val_losses = []
    opt_loss = np.inf
    count = 0
    
    if not regression:
        directory = data_info[0]
        class_mapping = data_info[1]
        read_spec_fn = data_info[2]
    
        train_loader, val_loader, test_loader = data_generator(directory, class_mapping, read_spec_fn)
        
    if regression:
        directory, regression_type = data_info
        train_loader, val_loader = get_multitask_dataset(directory,
                                                         dataset.CLASS_MAPPING,
                                                         regression=regression_type)
        

    for epoch in range(epochs):
        train_loss = one_epoch_train(model, train_loader, criterion, optimizer)
        val_loss, (y_gold, y_pred) = evaluate_model(model, val_loader, criterion, regression)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        if epoch%5 == 0:
            print(f'Epoch {epoch}: \t Training Loss = {train_loss} \t--\t Validation Loss = {val_loss}')

        
        if val_loss < opt_loss:
            #torch.save(model, "./"+model_name) # checkpoint
            opt_loss = val_loss
            opt_pred = y_pred
            count = 0
        else:
            count += 1

        if early_stopping:
            if count == 8:  # if it doesn't get any better, stop so as not to overfit
                print(' !!-- Terminated Training due to Early Stopping --!! ')
                break
           
    # free up some space
    del train_loader
    del val_loader
    
    if not regression:
        del test_loader
    gc.collect()

    if regression:
        return train_losses, val_losses, y_gold, opt_pred
    
    return train_losses, val_losses

In [30]:
data_info_mel         = (fma_genre_spectrograms, dataset.CLASS_MAPPING, dataset.read_mel_spectrogram)
data_info_mel_beat    = (fma_genre_spectrograms_beat, dataset.CLASS_MAPPING, dataset.read_mel_spectrogram)
data_info_chroma      = (fma_genre_spectrograms, dataset.CLASS_MAPPING, dataset.read_chromagram)
data_info_chroma_beat = (fma_genre_spectrograms_beat, dataset.CLASS_MAPPING, dataset.read_chromagram)
data_info_fused       = (fma_genre_spectrograms, dataset.CLASS_MAPPING, dataset.read_fused_spectrogram)

First, we intentionally overfit a model to check whether our architecture is trainable and free of bugs.

In [150]:
# intentionally overfit the model to check for bugs and trainability
model_overfit = lab2_lstm.CustomLSTM(input_dim=n_mel,
                                     rnn_size=RNN_SIZE,
                                     output_dim=n_classes,
                                     num_layers=num_layers,
                                     bidirectional=True,
                                     dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_overfit.parameters(), lr=lr)

In [151]:
# train the model so as to overfit and check trainability
train_model(model=model_overfit,
            data_info=data_info_mel_beat,
            criterion=criterion,
            optimizer=optimizer,
            epochs=8000,
            early_stopping=False,
            overfit_batch=True)

### Training a model on mel spectrograms (non-beat-synced)

In [182]:
model_mel = lab2_lstm.CustomLSTM(input_dim=n_mel,
                                 rnn_size=RNN_SIZE,
                                 output_dim=n_classes,
                                 num_layers=num_layers,
                                 bidirectional=True,
                                 dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_mel.parameters(), lr=lr)

In [183]:
train_losses_mel, val_losses_mel = train_model(model=model_mel,
                                               data_info=data_info_mel,
                                               criterion=criterion,
                                               optimizer=optimizer,
                                               epochs=EPOCHS,
                                               early_stopping=False,
                                               overfit_batch=False)

In [184]:
plt.figure(figsize=(8,6))
plt.plot(train_losses_mel, label="Train Loss", color="#3498db")
plt.plot(val_losses_mel, label="Validation Loss", color="#e74c3c")
plt.legend()
plt.show()

### Training a model on beat-synced mel spectrograms

In [185]:
model_mel_beat = lab2_lstm.CustomLSTM(input_dim=n_mel,
                                      rnn_size=RNN_SIZE,
                                      output_dim=n_classes,
                                      num_layers=num_layers,
                                      bidirectional=True,
                                      dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_mel_beat.parameters(), lr=lr)

In [186]:
train_losses_mel_beat, val_losses_mel_beat = train_model(model=model_mel_beat,
                                                         data_info=data_info_mel_beat,
                                                         criterion=criterion,
                                                         optimizer=optimizer,
                                                         epochs=EPOCHS,
                                                         early_stopping=False,
                                                         overfit_batch=False)

In [187]:
plt.figure(figsize=(8,6))
plt.plot(train_losses_mel_beat, label="Train Loss", color="#3498db")
plt.plot(val_losses_mel_beat, label="Validation Loss", color="#e74c3c")
plt.legend()
plt.show()

### Training a model on chromagrams (non-beat-synced)

In [188]:
model_chroma = lab2_lstm.CustomLSTM(input_dim=n_chroma,
                                    rnn_size=RNN_SIZE,
                                    output_dim=n_classes,
                                    num_layers=num_layers,
                                    bidirectional=True,
                                    dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_chroma.parameters(), lr=lr)

In [189]:
train_losses_chroma, val_losses_chroma = train_model(model=model_chroma,
                                                     data_info=data_info_chroma,
                                                     criterion=criterion,
                                                     optimizer=optimizer,
                                                     epochs=EPOCHS*2,
                                                     early_stopping=False,
                                                     overfit_batch=False)

In [190]:
plt.figure(figsize=(8,6))
plt.plot(train_losses_chroma, label="Train Loss", color="#3498db")
plt.plot(val_losses_chroma, label="Validation Loss", color="#e74c3c")
plt.legend()
plt.show()

### Training a model on fused data (non-beat-synced)

In [191]:
model_fused = lab2_lstm.CustomLSTM(input_dim=n_mel+n_chroma,
                                   rnn_size=RNN_SIZE,
                                   output_dim=n_classes,
                                   num_layers=num_layers,
                                   bidirectional=True,
                                   dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_fused.parameters(), lr=lr)

In [192]:
train_losses_fused, val_losses_fused = train_model(model=model_fused,
                                                   data_info=data_info_fused,
                                                   criterion=criterion,
                                                   optimizer=optimizer,
                                                   epochs=EPOCHS,
                                                   early_stopping=False,
                                                   overfit_batch=False)

In [193]:
plt.figure(figsize=(8,6))
plt.plot(train_losses_fused, label="Train Loss", color="#3498db")
plt.plot(val_losses_fused, label="Validation Loss", color="#e74c3c")
plt.legend()
plt.show()

## Step 6: Model Evaluation

### Evaluate the model trained on non-beat-synced mel spectrograms

In [194]:
train_loader, val_loader, test_loader = data_generator(fma_genre_spectrograms,
                                                       dataset.CLASS_MAPPING,
                                                       dataset.read_mel_spectrogram)

test_loss, (y_test_gold, y_test_pred) = evaluate_model(model=model_mel,
                                                       dataloader=test_loader,
                                                       criterion=criterion)

y_test_true = np.concatenate(y_test_gold, axis=0)
y_test_pred = np.concatenate(y_test_pred, axis=0)

print(classification_report(y_test_true, y_test_pred))

del train_loader
del val_loader
del test_loader
gc.collect()

## Evaluate the model trained on beat-synced mel spectrograms

In [195]:
train_loader, val_loader, test_loader = data_generator(fma_genre_spectrograms_beat,
                                                       dataset.CLASS_MAPPING,
                                                       dataset.read_mel_spectrogram)

test_loss, (y_test_gold, y_test_pred) = evaluate_model(model=model_mel_beat,
                                                       dataloader=test_loader,
                                                       criterion=criterion)

y_test_true = np.concatenate(y_test_gold, axis=0)
y_test_pred = np.concatenate(y_test_pred, axis=0)

print(classification_report(y_test_true, y_test_pred))

del train_loader
del val_loader
del test_loader
gc.collect()

## Evaluate the model trained on chromagrams

In [196]:
train_loader, val_loader, test_loader = data_generator(fma_genre_spectrograms,
                                                       dataset.CLASS_MAPPING,
                                                       dataset.read_chromagram)

test_loss, (y_test_gold, y_test_pred) = evaluate_model(model=model_chroma,
                                                       dataloader=test_loader,
                                                       criterion=criterion)

y_test_true = np.concatenate(y_test_gold, axis=0)
y_test_pred = np.concatenate(y_test_pred, axis=0)

print(classification_report(y_test_true, y_test_pred))

del train_loader
del val_loader
del test_loader
gc.collect()

### Evaluate the model trained on fused data

In [197]:
train_loader, val_loader, test_loader = data_generator(fma_genre_spectrograms,
                                                       dataset.CLASS_MAPPING,
                                                       dataset.read_fused_spectrogram)

test_loss, (y_test_gold, y_test_pred) = evaluate_model(model=model_fused,
                                                       dataloader=test_loader,
                                                       criterion=criterion)

y_test_true = np.concatenate(y_test_gold, axis=0)
y_test_pred = np.concatenate(y_test_pred, axis=0)

print(classification_report(y_test_true, y_test_pred))

del train_loader
del val_loader
del test_loader
gc.collect()

## Re-training using Early Stopping

We will now train the same models as before but using early stopping.

In [201]:
## Non-beat-synced Spectrograms - Early Stopping
# definition
model_mel_early = lab2_lstm.CustomLSTM(input_dim=n_mel,
                                       rnn_size=RNN_SIZE,
                                       output_dim=n_classes,
                                       num_layers=num_layers,
                                       bidirectional=True,
                                       dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_mel_early.parameters(), lr=lr)

# training
train_losses_mel_e, val_losses_mel_e = train_model(model=model_mel_early,
                                                   data_info=data_info_mel,
                                                   criterion=criterion,
                                                   optimizer=optimizer,
                                                   epochs=EPOCHS,
                                                   early_stopping=True,
                                                   overfit_batch=False)

# plotting
plt.figure(figsize=(8,6))
plt.plot(train_losses_mel_e, label="Train Loss", color="#3498db")
plt.plot(val_losses_mel_e, label="Validation Loss", color="#e74c3c")
plt.legend()
plt.show()

In [205]:
## Beat-synced Spectrograms - Early Stopping
# definition
model_mel_beat_early = lab2_lstm.CustomLSTM(input_dim=n_mel,
                                            rnn_size=RNN_SIZE,
                                            output_dim=n_classes,
                                            num_layers=num_layers,
                                            bidirectional=True,
                                            dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_mel_beat_early.parameters(), lr=lr)

# training
train_losses_mel_beat_e, val_losses_mel_beat_e = train_model(model=model_mel_beat_early,
                                                             data_info=data_info_mel_beat,
                                                             criterion=criterion,
                                                             optimizer=optimizer,
                                                             epochs=EPOCHS,
                                                             early_stopping=True,
                                                             overfit_batch=False)

# plotting
plt.figure(figsize=(8,6))
plt.plot(train_losses_mel_beat_e, label="Train Loss", color="#3498db")
plt.plot(val_losses_mel_beat_e, label="Validation Loss", color="#e74c3c")
plt.legend()
plt.show()

In [206]:
## Non-beat-synced Chromagrams - Early Stopping
# definition
model_chroma_early = lab2_lstm.CustomLSTM(input_dim=n_chroma,
                                          rnn_size=RNN_SIZE,
                                          output_dim=n_classes,
                                          num_layers=num_layers,
                                          bidirectional=True,
                                          dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_chroma_early.parameters(), lr=lr)

# training
train_losses_chroma_e, val_losses_chroma_e = train_model(model=model_chroma_early,
                                                     data_info=data_info_chroma,
                                                     criterion=criterion,
                                                     optimizer=optimizer,
                                                     epochs=EPOCHS*2,
                                                     early_stopping=True,
                                                     overfit_batch=False)

# plotting
plt.figure(figsize=(8,6))
plt.plot(train_losses_chroma_e, label="Train Loss", color="#3498db")
plt.plot(val_losses_chroma_e, label="Validation Loss", color="#e74c3c")
plt.legend()
plt.show()

In [207]:
## Non-beat-synced Fused Spectrogram - Early Stopping
# definition
model_fused_early = lab2_lstm.CustomLSTM(input_dim=n_mel+n_chroma,
                                         rnn_size=RNN_SIZE,
                                         output_dim=n_classes,
                                         num_layers=num_layers,
                                         bidirectional=True,
                                         dropout=0).double().to(DEVICE)

optimizer = torch.optim.Adam(model_fused_early.parameters(), lr=lr)

# training
train_losses_fused_e, val_losses_fused_e = train_model(model=model_fused_early,
                                                   data_info=data_info_fused,
                                                   criterion=criterion,
                                                   optimizer=optimizer,
                                                   epochs=EPOCHS,
                                                   early_stopping=True,
                                                   overfit_batch=False)

# plotting
plt.figure(figsize=(8,6))
plt.plot(train_losses_fused_e, label="Train Loss", color="#3498db")
plt.plot(val_losses_fused_e, label="Validation Loss", color="#e74c3c")
plt.legend()
plt.show()

We will evaluate the models whose training was interrupted by early stopping.

In [210]:
## Non-beat-synced mel spectrograms Evaluation - Early Stopping
train_loader, val_loader, test_loader = data_generator(fma_genre_spectrograms,
                                                       dataset.CLASS_MAPPING,
                                                       dataset.read_mel_spectrogram)

test_loss, (y_test_gold, y_test_pred) = evaluate_model(model=model_mel_early,
                                                       dataloader=test_loader,
                                                       criterion=criterion)

y_test_true = np.concatenate(y_test_gold, axis=0)
y_test_pred = np.concatenate(y_test_pred, axis=0)

print(classification_report(y_test_true, y_test_pred))

del train_loader
del val_loader
del test_loader
gc.collect()

In [211]:
## Non-beat-synced Chromagrams Evaluation - Early Stopping
train_loader, val_loader, test_loader = data_generator(fma_genre_spectrograms,
                                                       dataset.CLASS_MAPPING,
                                                       dataset.read_chromagram)

test_loss, (y_test_gold, y_test_pred) = evaluate_model(model=model_chroma_early,
                                                       dataloader=test_loader,
                                                       criterion=criterion)

y_test_true = np.concatenate(y_test_gold, axis=0)
y_test_pred = np.concatenate(y_test_pred, axis=0)

print(classification_report(y_test_true, y_test_pred))

del train_loader
del val_loader
del test_loader
gc.collect()

In [212]:
## Non beat-synced Fused Spectrograms Evaluation - Early Stopping
train_loader, val_loader, test_loader = data_generator(fma_genre_spectrograms,
                                                       dataset.CLASS_MAPPING,
                                                       dataset.read_fused_spectrogram)

test_loss, (y_test_gold, y_test_pred) = evaluate_model(model=model_fused_early,
                                                       dataloader=test_loader,
                                                       criterion=criterion)

y_test_true = np.concatenate(y_test_gold, axis=0)
y_test_pred = np.concatenate(y_test_pred, axis=0)

print(classification_report(y_test_true, y_test_pred))

del train_loader
del val_loader
del test_loader
gc.collect()

At this point we restart kernel to free all allocated RAM